# Designing an ETL process for COVID-19 related protein data


In this Jupyter notebook, I builded a data science/engineering project that is aiming to extract biological data from different sources with different techniques. The extracted data was transformed (cleaning) using pandas and loaded into MySQL. A schema was created to connect the data.

***1. Data extraction from:***

***ChEMBL Data Web Services***
- Overview of covid-19 data
- Target data
- Activity data
- Assay data

***UniProt***
- Retrieve ID/mapping service: from ChEMBL to UniProtKB
- Download xlsx file

***2. Data transformation***
- Data cleaning
- Dropping columns
- Renaming columns 
- Dealing with missing values
- Remove the duplicated data

***3. Connecting python to MySQL and data loading***
- Use SQLAlchemy to connect python to MySQl
- Create a MySQL Schema

# 1. Data extraction

### Installing and Importing libraries

In [24]:
#!pip install chembl_webresource_client

In [25]:
# Import necessary libraries
import pandas as pd #import pandas library 
import numpy as np #import pandas library 
from chembl_webresource_client.new_client import new_client #Import ChEMBL python client
import sqlalchemy #import sqlalchemy

### Available data entities

In [179]:
available_resources = [resource for resource in dir(new_client) if not resource.startswith('_')]
print(available_resources)

['activity', 'activity_supplementary_data_by_activity', 'assay', 'assay_class', 'atc_class', 'binding_site', 'biotherapeutic', 'cell_line', 'chembl_id_lookup', 'compound_record', 'compound_structural_alert', 'description', 'document', 'document_similarity', 'drug', 'drug_indication', 'drug_warning', 'go_slim', 'image', 'mechanism', 'metabolism', 'molecule', 'molecule_form', 'official', 'organism', 'protein_class', 'similarity', 'source', 'substructure', 'target', 'target_component', 'target_relation', 'tissue', 'xref_source']


### Search for Target protein,  Activity and Assay


#### Target

In [180]:
# Target search for covid-19 and get the overview target table (50 rows)
target = new_client.target
target_query = target.search('covid-19')
targets_overview = pd.DataFrame.from_dict(target_query)
targets_overview.head(2)

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'Q712U5', 'xref_name': None, 'xre...",Rattus norvegicus,cyclic AMP phosphoprotein,15.0,False,CHEMBL2170,"[{'accession': 'Q712U5', 'component_descriptio...",SINGLE PROTEIN,10116
1,[],Homo sapiens,KU-19-19,15.0,False,CHEMBL1075483,[],CELL-LINE,9606


In [181]:
#drop columns cross_references
targets_overview = targets_overview.drop(['cross_references', 'target_components'], axis=1)
targets_overview.head(2)

,organism,pref_name,score,species_group_flag,target_chembl_id,target_type,tax_id
0,Rattus norvegicus,cyclic AMP phosphoprotein,15.0,False,CHEMBL2170,SINGLE PROTEIN,10116
1,Homo sapiens,KU-19-19,15.0,False,CHEMBL1075483,CELL-LINE,9606


In [207]:
targets_overview.columns

Index(['organism', 'pref_name', 'score', 'species_group_flag',
       'target_chembl_id', 'target_type', 'tax_id'],
      dtype='object')

In [182]:
#get table info, the overview table must have 50 rows
targets_overview.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   organism            50 non-null     object 
 1   pref_name           50 non-null     object 
 2   score               50 non-null     float64
 3   species_group_flag  50 non-null     bool   
 4   target_chembl_id    50 non-null     object 
 5   target_type         50 non-null     object 
 6   tax_id              50 non-null     int64  
dtypes: bool(1), float64(1), int64(1), object(4)
memory usage: 2.5+ KB


In [183]:
# Target search from targets_overview and get the activity table
selected_target = targets_overview.target_chembl_id[19]
selected_target

'CHEMBL3883323'

#### Activity

In [188]:
activity = new_client.activity
res1 = activity.filter(target_chembl_id=selected_target)
activity = pd.DataFrame(res1)
activity.head(2)

,activity_comment,activity_id,activity_properties,assay_chembl_id,assay_description,assay_type,assay_variant_accession,assay_variant_mutation,bao_endpoint,bao_format,...,target_organism,target_pref_name,target_tax_id,text_value,toid,type,units,uo_units,upper_value,value
0,None,16508071,[],CHEMBL3779201,Binding affinity to CDK19/Cyclin C (unknown or...,B,None,None,BAO_0000190,BAO_0000223,...,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,None,None,IC50,nM,UO_0000065,None,26.0
1,None,16509067,[],CHEMBL3779201,Binding affinity to CDK19/Cyclin C (unknown or...,B,None,None,BAO_0000190,BAO_0000223,...,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,None,None,IC50,nM,UO_0000065,None,4.0


In [189]:
activity.columns

Index(['activity_comment', 'activity_id', 'activity_properties',
       'assay_chembl_id', 'assay_description', 'assay_type',
       'assay_variant_accession', 'assay_variant_mutation', 'bao_endpoint',
       'bao_format', 'bao_label', 'canonical_smiles', 'data_validity_comment',
       'data_validity_description', 'document_chembl_id', 'document_journal',
       'document_year', 'ligand_efficiency', 'molecule_chembl_id',
       'molecule_pref_name', 'parent_molecule_chembl_id', 'pchembl_value',
       'potential_duplicate', 'qudt_units', 'record_id', 'relation', 'src_id',
       'standard_flag', 'standard_relation', 'standard_text_value',
       'standard_type', 'standard_units', 'standard_upper_value',
       'standard_value', 'target_chembl_id', 'target_organism',
       'target_pref_name', 'target_tax_id', 'text_value', 'toid', 'type',
       'units', 'uo_units', 'upper_value', 'value'],
      dtype='object')

In [190]:
activity = activity.drop(['activity_comment', 'activity_properties', 'assay_variant_mutation', 'assay_variant_accession', 'canonical_smiles', 'data_validity_comment', 'data_validity_description', 'record_id', 'src_id', 'standard_upper_value', 'standard_text_value', 'text_value', 'toid', 'ligand_efficiency', 'uo_units', 'qudt_units', 'upper_value'], axis=1)
activity

,activity_id,assay_chembl_id,assay_description,assay_type,bao_endpoint,bao_format,bao_label,document_chembl_id,document_journal,document_year,...,standard_type,standard_units,standard_value,target_chembl_id,target_organism,target_pref_name,target_tax_id,type,units,value
0,16508071,CHEMBL3779201,Binding affinity to CDK19/Cyclin C (unknown or...,B,BAO_0000190,BAO_0000223,protein complex format,CHEMBL3774357,Bioorg. Med. Chem. Lett.,2016.0,...,IC50,nM,26.0,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,nM,26.0
1,16509067,CHEMBL3779201,Binding affinity to CDK19/Cyclin C (unknown or...,B,BAO_0000190,BAO_0000223,protein complex format,CHEMBL3774357,Bioorg. Med. Chem. Lett.,2016.0,...,IC50,nM,4.0,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,nM,4.0
2,16509069,CHEMBL3779202,Binding affinity to CDK19/Cyclin C (unknown or...,B,BAO_0000179,BAO_0000223,protein complex format,CHEMBL3774357,Bioorg. Med. Chem. Lett.,2016.0,...,TIME,hr,1.0,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,Time,min,60.0
3,16573757,CHEMBL3803437,Binding affinity to human CDK19 (1 to 502 amin...,B,BAO_0000190,BAO_0000223,protein complex format,CHEMBL3797072,J. Med. Chem.,2016.0,...,IC50,nM,2.5,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,nM,2.5
4,16573763,CHEMBL3803437,Binding affinity to human CDK19 (1 to 502 amin...,B,BAO_0000190,BAO_0000223,protein complex format,CHEMBL3797072,J. Med. Chem.,2016.0,...,IC50,nM,1.4,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,nM,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280,22941951,CHEMBL4742446,Inhibition of kinase tracer 236 binding to ful...,B,BAO_0000179,BAO_0000223,protein complex format,CHEMBL4742387,None,NaN,...,IC50,10^-9mol/L,1.41,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,10^-9mol/L,1.41
281,22941952,CHEMBL4742446,Inhibition of kinase tracer 236 binding to ful...,B,BAO_0000179,BAO_0000223,protein complex format,CHEMBL4742387,None,NaN,...,IC50,10^-9mol/L,1.41,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,10^-9mol/L,1.41
282,22941953,CHEMBL4742446,Inhibition of kinase tracer 236 binding to ful...,B,BAO_0000179,BAO_0000223,protein complex format,CHEMBL4742387,None,NaN,...,IC50,10^-7mol/L,1.48,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,10^-7mol/L,1.48
283,22970718,CHEMBL4770577,Inhibition of CDK19/Cyclin C (unknown origin) ...,B,BAO_0000179,BAO_0000223,protein complex format,CHEMBL4765336,Eur J Med Chem,2020.0,...,IC50,nM,15540.0,CHEMBL3883323,Homo sapiens,Cyclin-C/Cyclin-dependent kinase 19,9606,IC50,nM,15540.0


In [191]:
#deal with NAN
activity.isna().sum()

activity_id                    0
assay_chembl_id                0
assay_description              0
assay_type                     0
bao_endpoint                   0
bao_format                     0
bao_label                      0
document_chembl_id             0
document_journal             158
document_year                157
molecule_chembl_id             0
molecule_pref_name           282
parent_molecule_chembl_id      0
pchembl_value                204
potential_duplicate            0
relation                      31
standard_flag                  0
standard_relation             31
standard_type                  0
standard_units                28
standard_value                30
target_chembl_id               0
target_organism                0
target_pref_name               0
target_tax_id                  0
type                           0
units                         30
value                         30
dtype: int64

#### Assay

In [192]:
assay = new_client.assay
res = assay.filter(description__icontains='inhibit', assay_type='A')
assay = pd.DataFrame(res)
assay.head(3)

,assay_category,assay_cell_type,assay_chembl_id,assay_classifications,assay_organism,assay_parameters,assay_strain,assay_subcellular_fraction,assay_tax_id,assay_test_type,...,confidence_score,description,document_chembl_id,relationship_description,relationship_type,src_assay_id,src_id,target_chembl_id,tissue_chembl_id,variant_sequence
0,None,None,CHEMBL884521,[],Rattus norvegicus,[],None,None,10116.0,None,...,9,Inhibition of cytochrome P450 progesterone 15-...,CHEMBL1125500,Direct protein target assigned,D,None,1,CHEMBL3705,None,None
1,None,None,CHEMBL615148,[],Rattus norvegicus,[],None,None,10116.0,None,...,0,Inhibition of cytochrome P450 progesterone 16-...,CHEMBL1125500,Default value - Target has yet to be curated,U,None,1,CHEMBL612558,None,None
2,None,None,CHEMBL615199,[],Rattus norvegicus,[],None,None,10116.0,None,...,9,Inhibition of cytochrome P450 progesterone 2-a...,CHEMBL1125500,Direct protein target assigned,D,None,1,CHEMBL4971,None,None


In [193]:
assay= assay.filter(["assay_chembl_id", "assay_organism", "assay_tax_id", "confidence_score", "description", "document_chembl_id", "relationship_type", "target_chembl_id"])
assay.head(3)

,assay_chembl_id,assay_organism,assay_tax_id,confidence_score,description,document_chembl_id,relationship_type,target_chembl_id
0,CHEMBL884521,Rattus norvegicus,10116.0,9,Inhibition of cytochrome P450 progesterone 15-...,CHEMBL1125500,D,CHEMBL3705
1,CHEMBL615148,Rattus norvegicus,10116.0,0,Inhibition of cytochrome P450 progesterone 16-...,CHEMBL1125500,U,CHEMBL612558
2,CHEMBL615199,Rattus norvegicus,10116.0,9,Inhibition of cytochrome P450 progesterone 2-a...,CHEMBL1125500,D,CHEMBL4971


### Importing xlsx files

#### Covid-19 data from Chembl

In [194]:
#import xlsx from chembl which have a connexion to uniprot database
#load xlsx from chembl
covid_chembl = pd.read_excel('covid-19-chembl.xlsx')
covid_chembl.head(2)

,ChEMBL ID,Name,UniProt Accessions,Type,Organism,Compounds,Activities,Tax ID,Species Group Flag
0,CHEMBL4630723,ErbB-2/ErbB-3 heterodimer,P04626|P21860,PROTEIN COMPLEX,Homo sapiens,NaN,NaN,9606,False
1,CHEMBL3831285,Interleukin 13 receptor,P24394|P78552,PROTEIN COMPLEX,Homo sapiens,NaN,NaN,9606,False


In [195]:
covid_chembl.rename(columns={"ChEMBL ID": "chembl_id"}, inplace = True)

In [196]:
#covid_chembl

In [197]:
covid_chembl.isna().sum()

chembl_id             0
Name                  0
UniProt Accessions    5
Type                  0
Organism              0
Compounds             2
Activities            2
Tax ID                0
Species Group Flag    0
dtype: int64

In [199]:
#covid_chembl.duplicated()

#### Covid-19 data from Uniprot

In [200]:
# load xlsx from uniprot
uniprot =pd.read_excel('uniprot-covid-19-chembel_mapped.xlsx')
uniprot.head(2)

#_______________ALTERNATIVE_______________________Import uniprot ID/mapping with API_______________________________________
#import requests, json, sys
# Helper function to download data
#def get_url(url, **kwargs):
#    response = requests.get(url, **kwargs);

#    if not response.ok:
#        print(response.text)
#        response.raise_for_status()
#        sys.exit()

#    return response

#url = "https://rest.uniprot.org/idmapping/uniprotkb/results/stream/19ab23d37132d4b2cf08652174334af86efd2882?format=json"
#r = get_url(url)
#print(json.dumps(r.json(), indent=2))

,From,Entry,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Active site,Developmental stage,Interacts with,Sequence
0,CHEMBL3831285,P78552,reviewed,I13R1_HUMAN,"Interleukin-13 receptor subunit alpha-1, IL-13...",IL13RA1 IL13R IL13RA,Homo sapiens (Human),427,NaN,NaN,P35225; P05112; Q8TDR0,MEWPARLCGLWALLLCAGGGGGGGGAAPTETQPPVTNLSVSVENLC...
1,CHEMBL3079,O60779,reviewed,S19A2_HUMAN,"Thiamine transporter 1, ThTr-1, ThTr1 (Solute ...",SLC19A2 THT1 TRMA,Homo sapiens (Human),497,NaN,NaN,O60635,MDVPGPVSRRAAAAAATVLLRTARVRRECWFLPTALLCAYGFFASL...


In [141]:
#renaming the Column 'ID' for unprot and chembl
uniprot.rename(columns={"From": "chembl_id", "Entry": "uniprot_id"}, inplace = True)
uniprot.head(2)

,chembl_id,uniprot_id,Reviewed,Entry Name,Protein names,Gene Names,Organism,Length,Active site,Developmental stage,Interacts with,Sequence
0,CHEMBL3831285,P78552,reviewed,I13R1_HUMAN,"Interleukin-13 receptor subunit alpha-1, IL-13...",IL13RA1 IL13R IL13RA,Homo sapiens (Human),427,NaN,NaN,P35225; P05112; Q8TDR0,MEWPARLCGLWALLLCAGGGGGGGGAAPTETQPPVTNLSVSVENLC...
1,CHEMBL3079,O60779,reviewed,S19A2_HUMAN,"Thiamine transporter 1, ThTr-1, ThTr1 (Solute ...",SLC19A2 THT1 TRMA,Homo sapiens (Human),497,NaN,NaN,O60635,MDVPGPVSRRAAAAAATVLLRTARVRRECWFLPTALLCAYGFFASL...


In [128]:
uniprot.isna().sum()

chembl_id               0
uniprot_id              0
Reviewed                0
Entry Name              0
Protein names           0
Gene Names              0
Organism                0
Length                  0
Active site            66
Developmental stage    75
Interacts with         30
Sequence                0
dtype: int64

In [133]:
uniprot.duplicated()

0     False
1     False
2     False
3     False
4     False
      ...  
72    False
73    False
74    False
75    False
76    False
Length: 77, dtype: bool

# 3. Connecting python to MySQL with SQLAlchemy

In [13]:
!pip install sqlalchemy

In [152]:
# connecting python to mysql workbench
schema="Covid_19"
host="127.0.0.1"
user="root"
password="MY_PASSWORD"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [250]:
# load the uniprot dataframe in mysql

(
uniprot
    .replace({np.nan},'unknown')
    .to_sql('covid_uniprot', if_exists='append', con=con, index=False)
)

OperationalError: (pymysql.err.OperationalError) (1054, "Unknown column 'From' in 'field list'")
[SQL: INSERT INTO covid_uniprot (`From`, `Entry`, `Reviewed`, `Entry Name`, `Protein names`, `Gene Names`, `Organism`, `Length`, `Active site`, `Developmental stage`, `Interacts with`, `Sequence`) VALUES (%(From)s, %(Entry)s, %(Reviewed)s, %(Entry Name)s, %(Protein names)s, %(Gene Names)s, %(Organism)s, %(Length)s, %(Active site)s, %(Developmental stage)s, %(Interacts with)s, %(Sequence)s)]
[parameters: ({'From': 'CHEMBL3831285', 'Entry': 'P78552', 'Reviewed': 'reviewed', 'Entry Name': 'I13R1_HUMAN', 'Protein names': 'Interleukin-13 receptor subunit alpha-1, IL-13 receptor subunit alpha-1, IL-13R subunit alpha-1, IL-13R-alpha-1, IL-13RA1 (Cancer/testis antigen 19, CT19) (CD antigen CD213a1)', 'Gene Names': 'IL13RA1 IL13R IL13RA', 'Organism': 'Homo sapiens (Human)', 'Length': 427, 'Active site': 'unknown', 'Developmental stage': 'unknown', 'Interacts with': 'P35225; P05112; Q8TDR0', 'Sequenc

In [154]:
# load the covid_chembl dataframe in mysql

covid_chembl.to_sql('covid_chembl', con=con, if_exists='append', index=False)

In [155]:
# load the assay dataframe in mysql

assay.to_sql('assay', con=con, if_exists='append', index=False)

In [204]:
# load the activity dataframe in mysql
(
     activity.copy()
    .replace({np.nan},'unknown')
    #.assign(year = lambda x: pd.to_datetime(x['document_year']))
    .to_sql('activity', if_exists='append', con=con, index=False)
)


In [157]:
# load the targets_overview dataframe in mysql

targets_overview.to_sql('targets_overview', con=con, if_exists='append', index=False)